# Optimization and Best Practices

**Training Objective:** Mastering query and Delta table performance optimization techniques in Databricks.

**Topics covered:**
- Query optimization: predicate pushdown, file pruning, column pruning
- Physical plan analysis (explain())
- Table optimization: partitioning, small files problem
- Auto optimize / auto compaction
- File sizing and ZORDER strategy
- Liquid Clustering - modern alternative to partitioning


## Context and Requirements

- **Training Day**: Day 2 - Delta Lake & Lakehouse
- **Notebook Type**: Demo
- **Technical Requirements**:
  - Databricks Runtime 16.4 LTS or newer (recommended: 17.3 LTS)
  - Unity Catalog enabled
  - Permissions: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Cluster: Standard with minimum 2 workers or **Serverless Compute** (recommended)
- **Dependencies**: Completed notebook `01_delta_lake_operations.ipynb`
- **Duration**: ~45 minutes

> **Note (2025):** Serverless Compute is now the default mode for new workloads.


## Theoretical Introduction

**Section Objective:** Understanding key optimization mechanisms in Databricks and Delta Lake.

**Optimization Types:**

**1. Query Optimization:**
- **Predicate Pushdown**: Pushing filters as low as possible in the execution plan
- **Column Pruning**: Reading only required columns (Parquet columnar format)
- **File Pruning**: Skipping files irrelevant to the query
- **Join Optimization**: Broadcast joins, bucket joins, sortmerge joins

**2. Table Optimization:**
- **Partitioning**: Physical data separation by key values
- **Z-Ordering**: Data clustering in files by selected columns
- **Compaction (OPTIMIZE)**: Merging small files into larger ones
- **Auto Compaction**: Automatic merging during write

**3. Small Files Problem:**
Performance issue caused by too many small files in the table. Spark prefers 128MB-1GB files for optimal performance.


## Per-user Isolation

Run the initialization script for per-user catalog and schema isolation:


In [0]:
%run ../00_setup

## Configuration

Import libraries and set environment variables:


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit, count, avg, sum, max, min
from pyspark.sql.types import *
import time

# Set catalog and schema as default
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")


**User Context:**


In [0]:
display(
    spark.createDataFrame([
        ("CATALOG", CATALOG),
        ("BRONZE_SCHEMA", BRONZE_SCHEMA),
        ("SILVER_SCHEMA", SILVER_SCHEMA),
        ("GOLD_SCHEMA", GOLD_SCHEMA),
        ("USER", raw_user)
    ], ["Variable", "Value"])
)

## Section 0: Data Preparation

This notebook is **fully independent** - it loads source data and creates tables needed for the optimization demo.


In [0]:
# Source data paths
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
PRODUCTS_PARQUET = f"{DATASET_BASE_PATH}/products/products.parquet"

# Table names (unique for this notebook)
ORDERS_OPT = f"{BRONZE_SCHEMA}.orders_optimization"
CUSTOMERS_OPT = f"{BRONZE_SCHEMA}.customers_optimization"


**Load source data:**


In [0]:
# Load customers
customers_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(CUSTOMERS_CSV)

# Load orders
orders_df = spark.read.json(ORDERS_JSON)

# Add order_date column (date without time) for partitioning
orders_df = orders_df.withColumn(
    "order_date", 
    F.to_date(F.col("order_datetime"))
)

print(f"✓ Customers: {customers_df.count()} records")
print(f"✓ Orders: {orders_df.count()} records")


**Save as Delta tables for optimization:**


In [0]:
# Save as Delta tables
customers_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(CUSTOMERS_OPT)

orders_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(ORDERS_OPT)

display(spark.createDataFrame([
    ("CUSTOMERS_OPT", CUSTOMERS_OPT, str(customers_df.count())),
    ("ORDERS_OPT", ORDERS_OPT, str(orders_df.count()))
], ["Table", "Full Name", "Records"]))


In [0]:
%skip
orders_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("path", f"{DATASET_BASE_PATH}/delta/orders_optimization") \
    .saveAsTable(ORDERS_OPT)

## Section 1: Physical Plan Analysis (explain())

**Section Objective:** Learn to analyze query execution plans to identify performance bottlenecks.

**Theory:**
A physical plan is a detailed map of how Spark executes a query:
- **Stages**: Logical processing steps
- **Tasks**: Units of work executed on partitions
- **Shuffles**: Data exchange between executors
- **Pushdowns**: Optimizations pushed to the data source

**Types of explain():**
- `explain()` - basic plan
- `explain(True)` - full plan with details
- `explain('extended')` - extended plan
- `explain('cost')` - plan with cost


### Example 1.1: Simple Query Plan Analysis


In [0]:
# Example 1.1 - Simple Query Plan Analysis

simple_query = spark.sql(f"""
    SELECT customer_id, first_name, last_name, customer_segment, city
    FROM {CUSTOMERS_OPT}
    WHERE customer_segment = 'Premium'
    ORDER BY customer_id DESC
    LIMIT 10
""")


**Basic query plan:**


In [0]:
simple_query.explain()

**Extended query plan (with details):**


In [0]:
simple_query.explain(True)

### Example 1.2: Predicate Pushdown in Practice

**Theory:** Predicate pushdown is an optimization where filters (WHERE conditions) are "pushed" as low as possible in the execution plan, preferably to the file reading level. This way we only read data that meets the conditions.


In [0]:
# Example 1.2 - Predicate Pushdown

filtered_query = spark.sql(f"""
    SELECT order_id, customer_id, total_amount, order_date
    FROM {ORDERS_OPT}
    WHERE total_amount > 100 
    AND order_date >= '2024-01-01'
""")


**Check plan - look for "PushedFilters" in the plan:**


In [0]:
# Check plan - look for "PushedFilters" in the plan
filtered_query.explain(True)


**💡 In the plan look for:**
- `PushedFilters` - filters pushed to the reading level
- `ReadSchema` - only selected columns (column pruning)  
- `PartitionFilters` - filters on partitions


## Section 2: Partitioning Strategy

**Section Objective:** Learn to choose optimal partitioning keys for best performance.

**Partitioning Theory:**
- **Partitioning**: Physical separation of table into directories by column values
- **Partition Pruning**: Spark skips entire partitions that are not needed for the query
- **Ideal Partitions**: 1-10GB of data per partition, no more than 10,000 partitions

**Best Practices:**
- Partition by columns frequently used in filters
- Avoid partitioning by high cardinality columns
- Prefer columns with natural time hierarchy (year/month/day)
- Avoid too many small partitions (small files problem)


### Example 2.1: Creating a Partitioned Table


In [0]:
# Example 2.1 - Creating a Partitioned Table

# Create table partitioned by year and month
ORDERS_PARTITIONED = f"{BRONZE_SCHEMA}.orders_opt_partitioned"

# Add columns for partitioning
orders_with_partitions = spark.sql(f"""
    SELECT 
        *,
        YEAR(order_date) as year,
        MONTH(order_date) as month
    FROM {ORDERS_OPT}
""")


**Save as partitioned table:**


In [0]:
orders_with_partitions.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .saveAsTable(ORDERS_PARTITIONED)

**Check partition structure:**


In [0]:
display(
    spark.sql(f"DESCRIBE DETAIL {ORDERS_PARTITIONED}")
    .select("name", "location", "partitionColumns")
)

### Example 2.2: Partition Pruning in Action


In [0]:
# Example 2.2 - Partition Pruning

# Query that uses partitions (year/month)
efficient_query = spark.sql(f"""
    SELECT order_id, customer_id, total_amount, order_date
    FROM {ORDERS_PARTITIONED}
    WHERE year = 2024 AND month = 1
""")


**Check plan - partition pruning in action:**


In [0]:
# Check plan - look for "PartitionFilters"
efficient_query.explain(True)


**Comparison: query WITHOUT partition pruning:**


In [0]:
# Query that does not use partitions (does not filter by year/month)
inefficient_query = spark.sql(f"""
    SELECT order_id, customer_id, total_amount, order_date
    FROM {ORDERS_PARTITIONED}
    WHERE customer_id = 1
""")

inefficient_query.explain(True)


## Section 2b: JOIN Optimization and Shuffle

**Section Objective:** Optimizing JOIN operations and minimizing costly shuffle operations.

**Theory - JOIN Types in Spark:**

| JOIN Type | When Used | Characteristics |
|-----------|-----------|-----------------|
| **Broadcast Hash Join** | One table < 10MB (default) | Fastest - small table copied to all executors |
| **Sort Merge Join** | Both tables large | Requires shuffle and sorting of both sides |
| **Shuffle Hash Join** | Medium tables | Shuffle without sorting |

**Shuffle - what is it?**
- Data exchange between executors (over network)
- Most expensive operation in Spark (I/O, network, serialization)
- Causes "stage boundaries" in DAG

**How to minimize shuffle:**
1. **Broadcast JOIN** - for small tables (< 10MB, can be increased to 100MB)
2. **Repartition** before JOIN - aligning partitions
3. **Colocated JOIN** - data already on the same partitions
4. **Bucketing** - table pre-partitioning


### Example 2b.1: Broadcast JOIN - optimization for small tables


In [0]:
# Example 2b.1 - Broadcast JOIN

from pyspark.sql.functions import broadcast

# Load tables for JOIN
orders_df = spark.table(ORDERS_OPT)
customers_df = spark.table(CUSTOMERS_OPT)

print(f"Orders: {orders_df.count()} records")
print(f"Customers: {customers_df.count()} records (small table - candidate for broadcast)")


**Standard JOIN (no broadcast) - requires shuffle:**


In [0]:
# Standard JOIN - Spark decides strategy
standard_join = orders_df.join(
    customers_df,
    orders_df.customer_id == customers_df.customer_id,
    "inner"
).select(
    orders_df.order_id,
    orders_df.total_amount,
    customers_df.first_name,
    customers_df.customer_segment
)

# Check plan - look for "SortMergeJoin" or "BroadcastHashJoin"
standard_join.explain()


**Forced Broadcast JOIN - eliminates shuffle:**


In [0]:
# Forced Broadcast JOIN - small table (customers) copied to all executors
broadcast_join = orders_df.join(
    broadcast(customers_df),  # Force broadcast of smaller table
    orders_df.customer_id == customers_df.customer_id,
    "inner"
).select(
    orders_df.order_id,
    orders_df.total_amount,
    customers_df.first_name,
    customers_df.customer_segment
)

# Check plan - should show "BroadcastHashJoin"
broadcast_join.explain()


**💡 In the plan look for:**
- `BroadcastHashJoin` - broadcast join (no shuffle!)
- `SortMergeJoin` - sort merge join (requires shuffle of both sides)
- `BroadcastExchange` - copying small table to executors
- `ShuffleExchange` - shuffle (expensive operation!)


### Example 2b.3: Repartition vs Coalesce - shuffle control

**What does it mean?**

- **repartition(n)** – causes full data shuffle, distributing it evenly into n partitions. Use when you want to increase partition count or balance data distribution (e.g. before large JOIN).
- **coalesce(n)** – merges existing partitions without shuffle (narrow transformation), reducing their count. Use when you want to reduce partition count (e.g. before writing to file), but don't care about even distribution.

**Summary:**  
Choose `repartition` when you care about even distribution and can accept shuffle cost. Choose `coalesce` when you just want to reduce partition count without expensive shuffle.


In [0]:
# Repartition vs Coalesce - differences

# REPARTITION - full shuffle, even data distribution
# Use when: need more partitions OR even distribution
orders_repartitioned = orders_df.repartition(10)
print(f"After repartition(10): {orders_repartitioned.rdd.getNumPartitions()} partitions")

# COALESCE - no shuffle, just merging partitions (narrow transformation)
# Use when: reducing partition count (e.g. before write)
orders_coalesced = orders_df.coalesce(4)
print(f"After coalesce(4): {orders_coalesced.rdd.getNumPartitions()} partitions")


**Repartition BY column - optimization for JOIN:**


In [0]:
# Repartition BY join key - data with same key goes to same partition
# This minimizes shuffle during JOIN (collocated data)

orders_by_customer = orders_df.repartition(10, "customer_id")
customers_by_id = customers_df.repartition(10, "customer_id")

# Now JOIN will be faster - data is already on same partitions
optimized_join = orders_by_customer.join(
    customers_by_id,
    "customer_id",  # Common partition column
    "inner"
)

print("✓ Data repartitioned by customer_id - JOIN will be faster")
optimized_join.explain()


### Example 2b.4: JOIN Performance Comparison

**Theory - when to use which JOIN type:**

| Scenario | Recommendation |
|----------|----------------|
| Small lookup table (< 10MB) | `broadcast(small_df)` |
| Small lookup table (10-100MB) | Increase `autoBroadcastJoinThreshold` |
| Both tables large | SortMergeJoin (default) + pre-repartition |
| Frequent JOIN on same key | Bucketing (at table creation) |
| Skewed data (uneven distribution) | Salting or AQE (Adaptive Query Execution) |


In [0]:
# Performance Comparison: Broadcast vs SortMerge JOIN

import time

# Test 1: Broadcast JOIN
start_broadcast = time.time()
result_broadcast = orders_df.join(
    broadcast(customers_df),
    orders_df.customer_id == customers_df.customer_id
).count()
time_broadcast = time.time() - start_broadcast

# Test 2: Broadcast disabled (forces SortMergeJoin)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

start_sortmerge = time.time()
result_sortmerge = orders_df.join(
    customers_df,
    orders_df.customer_id == customers_df.customer_id
).count()
time_sortmerge = time.time() - start_sortmerge

# Restore default setting
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10 * 1024 * 1024)

# Results
display(spark.createDataFrame([
    ("Broadcast JOIN", f"{time_broadcast:.2f}s", "✅ Fast (no shuffle)"),
    ("SortMerge JOIN", f"{time_sortmerge:.2f}s", "⚠️ Slower (requires shuffle)")
], ["JOIN Type", "Time", "Notes"]))


### Example 2b.5: Shuffle Partitions - configuration


In [0]:
# Shuffle Partitions - controlling partition count after shuffle

# Check current value (default 200)
current_partitions = spark.conf.get("spark.sql.shuffle.partitions")
print(f"Current shuffle.partitions: {current_partitions}")

# For small data (< 1GB) - decrease partition count
# spark.conf.set("spark.sql.shuffle.partitions", 10)

# For large data (> 100GB) - increase
# spark.conf.set("spark.sql.shuffle.partitions", 500)

# Best Practice: Adaptive Query Execution (AQE) automatically optimizes
print(f"AQE enabled: {spark.conf.get('spark.sql.adaptive.enabled')}")
print(f"AQE coalesce partitions: {spark.conf.get('spark.sql.adaptive.coalescePartitions.enabled')}")


**📊 JOIN and Shuffle Optimization Summary:**

| Technique | When to use | Benefit |
|-----------|-------------|---------|
| `broadcast(df)` | Small table < 100MB | Eliminates shuffle |
| `repartition(n, col)` | Before JOIN on large tables | Collocated data |
| `coalesce(n)` | Reducing partitions before write | No shuffle |
| `shuffle.partitions` | Adjusting to data size | Optimal parallelization |
| AQE (Adaptive) | Always enabled (default) | Auto-optimization |

> **💡 Best Practice:** In Databricks Runtime 14+ AQE is enabled by default and automatically optimizes shuffle partitions, broadcast thresholds and skewed joins.


## Section 3: Small Files Problem

**Section Objective:** Understanding and solving the small files problem in Delta Lake.

**What is Small Files Problem?**
- When a table has too many small files (< 128MB each)
- Spark prefers 128MB-1GB files for optimal performance
- Small files cause metadata overhead and reduce throughput

**Causes of small files:**
- Frequent INSERT writes in small batches
- High partitioning with small amount of data per partition
- Streaming with short trigger intervals

**Solutions:**
- **OPTIMIZE** - merges small files into larger ones
- **Auto Compaction** - automatic merging during write
- **Repartition** before write
- **Coalesce** for reducing partition count


### Example 3.1: Small Files Problem Simulation


In [0]:
# Example 3.1 - Small Files Problem Simulation

SMALL_FILES_TABLE = f"{BRONZE_SCHEMA}.small_files_demo"


**Simulate many small writes (each creates a separate file):**


In [0]:
for i in range(5):
    small_batch = spark.range(i*100, (i+1)*100).select(
        col("id"),
        (col("id") * 2).alias("value"),
        lit(f"batch_{i}").alias("batch_name")
    )
    
    small_batch.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable(SMALL_FILES_TABLE)


**Check file count:**


In [0]:
detail = spark.sql(f"DESCRIBE DETAIL {SMALL_FILES_TABLE}").collect()[0]

display(
    spark.createDataFrame([
        ("File count", str(detail['numFiles'])),
        ("Table size", f"{detail['sizeInBytes']} bytes"),
        ("Avg file size", f"{detail['sizeInBytes'] / detail['numFiles']:.0f} bytes"),
        ("Status", "⚠️ Problem: too many small files!")
    ], ["Metric", "Value"])
)


### Example 3.2: Solution - OPTIMIZE and Auto Compaction


In [0]:
# Example 3.2 - Small Files Problem Solution

# Run OPTIMIZE on table with small files
spark.sql(f"OPTIMIZE {SMALL_FILES_TABLE}")


**Check state after OPTIMIZE:**


In [0]:
detail_after = spark.sql(f"DESCRIBE DETAIL {SMALL_FILES_TABLE}").collect()[0]

display(
    spark.createDataFrame([
        ("File count (AFTER OPTIMIZE)", str(detail_after['numFiles'])),
        ("Table size", f"{detail_after['sizeInBytes']} bytes"),
        ("Avg file size", f"{detail_after['sizeInBytes'] / detail_after['numFiles']:.0f} bytes")
    ], ["Metric", "Value"])
)


**Enable Auto Compaction for future writes:**


In [0]:
spark.sql(f"""
    ALTER TABLE {SMALL_FILES_TABLE}
    SET TBLPROPERTIES (
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true'
    )
""")


**Check enabled Auto Compaction properties:**


In [0]:
properties = spark.sql(f"SHOW TBLPROPERTIES {SMALL_FILES_TABLE}").collect()
auto_props = [(p['key'], p['value']) for p in properties if 'autoOptimize' in p['key']]

display(spark.createDataFrame(auto_props, ["Property", "Value"]))


### Example 3.3: VACUUM - Removing old files

**Theory:**
VACUUM removes old files that are no longer needed (after DELETE, UPDATE, MERGE, OPTIMIZE operations). 
By default Delta Lake retains files for 7 days (168 hours) for Time Travel.

**⚠️ Warning:** After VACUUM you cannot use Time Travel to versions older than retention period!


In [0]:
# Check how many files can be removed (DRY RUN)
spark.sql(f"VACUUM {SMALL_FILES_TABLE} DRY RUN")


**Run VACUUM (remove old files):**

> **Note:** In production use default retention (7 days). Code below with `RETAIN 0 HOURS` is for demo only!


In [0]:
# Run VACUUM - remove old files
# In demo environment we disable retention check
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

# VACUUM with short retention (DEMO ONLY!)
spark.sql(f"""
    VACUUM {ORDERS_OPT} RETAIN 1 HOURS
""")

# Restore default setting
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "true")

print("✅ VACUUM executed - old files removed")


## Section 4: ZORDER BY - Advanced Clustering

**Section Objective:** Learning to use ZORDER BY to optimize queries with filters and joins.

**What is ZORDER BY?**
- Multi-dimensional clustering algorithm in Delta Lake
- Organizes data in files by values of selected columns
- Improves data skipping - skipping unnecessary files during reading
- Especially effective for columns frequently used in WHERE filters and JOINs

**When to use ZORDER:**
- Columns frequently filtered in queries
- Columns used in JOIN operations
- High-cardinality columns (many unique values)
- Maximum 3-4 columns (more = diminishing returns)

**ZORDER vs Partitioning:**
- Partitioning: physical separation into directories
- ZORDER: logical ordering within files (preserves single folder structure)


### Example 4.1: ZORDER BY for frequently filtered columns


In [0]:
# Run ZORDER BY on most frequently filtered columns
spark.sql(f"""
    OPTIMIZE {ORDERS_OPT}
    ZORDER BY (customer_id, order_date)
""")


### Example 4.2: Measuring ZORDER effectiveness - Data Skipping


In [0]:
# Example 4.2 - Measuring ZORDER effectiveness

import time

# Query using ZORDER columns
# customer_id is STRING (e.g. CUST000123), order_date is DATE
test_query = spark.sql(f"""
    SELECT COUNT(*) as cnt, AVG(total_amount) as avg_amount
    FROM {ORDERS_OPT}
    WHERE customer_id BETWEEN 'CUST000100' AND 'CUST000500'
    AND order_date >= '2024-06-01'
""")


**Execution time measurement:**


In [0]:
start_time = time.time()
result = test_query.collect()
elapsed = time.time() - start_time

display(
    spark.createDataFrame([
        ("Result", str(result[0])),
        ("Execution time", f"{elapsed:.2f}s")
    ], ["Metric", "Value"])
)


**Query plan - check data skipping:**


In [0]:
# Check query plan - data skipping
test_query.explain(True)


**💡 In the plan look for:**
- `numFilesTotal` vs `numFilesSelected` - how many files were skipped
- `metadata time` - metadata parsing time
- `files pruned` - data skipping statistics


## Section 5: Liquid Clustering - Future of Optimization

**Section Objective:** Learning Liquid Clustering - modern technique replacing Hive Partitioning and ZORDER.

**What is Liquid Clustering?**
It's a flexible data layout mechanism that:
- Does not require rigid directory structure (like Partitioning)
- Allows changing clustering keys without rewriting entire table
- Eliminates "Small Files" problem related to excessive partitioning
- Works incrementally (no need to optimize entire table at once)

**When to use?**
- Instead of partitioning for most new tables
- When partitioning keys have high cardinality
- When query patterns change over time


In [0]:
LIQUID_TABLE = f"{BRONZE_SCHEMA}.orders_opt_liquid"

# Create table using CLUSTER BY instead of PARTITIONED BY
spark.sql(f"""
CREATE OR REPLACE TABLE {LIQUID_TABLE}
CLUSTER BY (customer_id, order_date)
AS SELECT * FROM {ORDERS_OPT}
""")


**Check table properties:**


In [0]:
# Check table properties
display(spark.sql(f"DESCRIBE DETAIL {LIQUID_TABLE}").select("name", "clusteringColumns"))


### Example 5.2: Incremental Optimization

**Theory:**
Unlike ZORDER, which must recalculate entire partition/table, Liquid Clustering works incrementally. `OPTIMIZE` will organize only data that needs it (e.g. newly added), saving time and resources.


In [0]:
# Run OPTIMIZE - Liquid Clustering knows how to layout data based on table definition
spark.sql(f"OPTIMIZE {LIQUID_TABLE}")


**Check history to see CLUSTERING operation:**


In [0]:
display(
    spark.sql(f"DESCRIBE HISTORY {LIQUID_TABLE}")
    .select("version", "operation", "operationParameters")
    .limit(5)
)


### Comparison: Liquid Clustering vs Partitioning + ZORDER

| Feature | Partitioning + ZORDER | Liquid Clustering |
|---------|-----------------------|-------------------|
| **Configuration** | Requires careful selection of partition columns | Flexible `CLUSTER BY` |
| **Small Files** | Risk with excessive partitioning | Automatically managed |
| **Key Change** | Difficult (requires table rewrite) | Easy (`ALTER TABLE CLUSTER BY`) |
| **Optimization** | `OPTIMIZE ZORDER BY` (expensive) | `OPTIMIZE` (incremental) |
| **Skew Data** | Susceptible to data skew | Resistant to data skew |

**Recommendation:** Use Liquid Clustering for all new tables in Databricks Runtime 13.3+, unless you have specific reason to use partitioning (e.g. compatibility with older readers).


## Summary

### What we achieved:

- **Performance Analysis**: Reading and interpreting physical plans with `explain()`
- **Predicate Pushdown**: Identifying bottlenecks and pushed filters
- **Partitioning**: Partitioning strategy by frequently filtered columns
- **ZORDER BY**: Multi-dimensional clustering for 2-4 columns
- **Small Files Problem**: Solving via OPTIMIZE and Auto Compaction
- **Liquid Clustering**: Modern alternative to partitioning

### Key Takeaways:

| # | Rule |
|---|------|
| 1 | **Analyze before optimizing** - always `explain()` first |
| 2 | **Partitioning ≠ ZORDER** - different techniques for different cases |
| 3 | **Small files = performance killer** - regular OPTIMIZE |
| 4 | **ZORDER BY** - max 3-4 columns, choose most frequently filtered |
| 5 | **Liquid Clustering** - prefer for new tables in DBR 13.3+ |


## Troubleshooting - Performance Diagnosis

### 🔍 Common problems and solutions:

**Problem 1: Query runs very slow**
```python
# Diagnosis:
df.explain(True)  # Check execution plan
```
**Possible causes:**
- Missing filters - reading entire table
- Shuffle operations - high network traffic
- Skewed data - uneven data distribution
- Small files - too many small files

**Problem 2: "OutOfMemoryError" during JOIN**
**Solution:**
```python
# Increase partitions before JOIN
df1 = df1.repartition(200, "join_key")
df2 = df2.repartition(200, "join_key")

# Or use broadcast join for small tables
from pyspark.sql.functions import broadcast
result = large_df.join(broadcast(small_df), "key")
```

**Problem 3: Long write times to Delta**
**Solution:**
- Enable Auto Compaction
- Use `coalesce()` before write
- Avoid too high partitioning

**Problem 4: OPTIMIZE does not improve performance**
**Cause:** ZORDER BY is needed for specific query patterns
```python
# Instead of just OPTIMIZE:
OPTIMIZE table_name

# Use OPTIMIZE with ZORDER:
OPTIMIZE table_name ZORDER BY (frequently_filtered_columns)
```


## Cleanup

Opcjonalnie usuń tabele demo utworzone podczas ćwiczeń:

In [0]:
# Cleanup - usuń tabele demo utworzone w tym notebooku

# Odkomentuj poniższe linie aby usunąć tabele demo:

# spark.sql(f"DROP TABLE IF EXISTS {ORDERS_OPT}")
# spark.sql(f"DROP TABLE IF EXISTS {CUSTOMERS_OPT}")
# spark.sql(f"DROP TABLE IF EXISTS {ORDERS_PARTITIONED}")
# spark.sql(f"DROP TABLE IF EXISTS {SMALL_FILES_TABLE}")
# spark.sql(f"DROP TABLE IF EXISTS {LIQUID_TABLE}")

# print("✅ Wszystkie tabele demo usunięte")

print("ℹ️ Cleanup wyłączony (odkomentuj kod aby usunąć tabele demo)")